In [8]:
# packages
import os
import sys
import numpy as np
from datetime import datetime
from dask.distributed import Client

In [9]:
CASE_NAME='ETCs_Test_2022_10'
user='ad1803' # Username on GADI - if you are storing on /scratch/nf33/user
N_CORES=1 # Cores for your job to use (set to 100, then it will run on 100 cores and you will need 100 cores. Runs 1 file per core, so if you have 5 files you need max 5 cores)

In [10]:
# directory for TempestExtremes
os.environ['TEMPESTEXTREMESDIR']='/scratch/nf33/tempestextremes/bin'

# link to self-written packages
sys.path.append(f"/scratch/nf33/{user}/hk25-AusCyclones") # change to your director
from utils.tools import create_Node_dirstruct, write_to_filelist, clear_dir, generate_datetimes_months
from utils.nci_utils import get_GADI_ERA5_filename
from tempestextremes_utils.node_utils import run_detectNodes, run_stitchNodes

TempestExtremes allows parallel running with `mpi`

In [11]:
# set dask workers
client = Client(n_workers=N_CORES)

In [12]:
# base directory (change to your directory)
base_dir = f"/scratch/nf33/{user}/hk25-AusCyclones/ETC_workfolder/Test_month/"
case_dir, input_dir, detect_dir, stitch_dir = create_Node_dirstruct(base_dir,CASE_NAME)

Directory '/scratch/nf33/ad1803/hk25-AusCyclones/ETC_workfolder/Test_month/ETCs_Test_2022_10' already exists.
Directory '/scratch/nf33/ad1803/hk25-AusCyclones/ETC_workfolder/Test_month/ETCs_Test_2022_10/input' already exists.
Directory '/scratch/nf33/ad1803/hk25-AusCyclones/ETC_workfolder/Test_month/ETCs_Test_2022_10/detectNodes' already exists.
Directory '/scratch/nf33/ad1803/hk25-AusCyclones/ETC_workfolder/Test_month/ETCs_Test_2022_10/stitchNodes' already exists.


#### CAREFUL!!!!! The below commands will clear out all your results directories! Proceed with caution!! #####

In [13]:
### be very careful with this - it will delete everything in the directory!!!
clear_dir(input_dir)
clear_dir(detect_dir)
clear_dir(stitch_dir)

rm: cannot remove '/scratch/nf33/ad1803/hk25-AusCyclones/ETC_workfolder/Test_month/ETCs_Test_2022_10/input/input_files.txt': Permission denied


CalledProcessError: Command 'rm -f /scratch/nf33/ad1803/hk25-AusCyclones/ETC_workfolder/Test_month/ETCs_Test_2022_10/input/*' returned non-zero exit status 1.

**Required variables for TC detection**  

| Variable Name                 | Level (hPa)                       |
|-------------------------------|-----------------------------------|
| Elevation (zs)                | Surface                           |
| Mean Sea Level Pressure (msl) | Surface                           |
| Geopotential (z)              | 500, 300                          |

**Create lists for inputfile and outputfile**

Inputfile consist of several files containing geopotential height (z) on pressure surfaces, mean sea level pressure (msl), 10-m zonal and meridional wind speeds (u10 and v10), and surface elevation (zs), separated by semicolons. Note that surface elevation data can found at `~/data/zs_era5_oper_sfc_invariant.nc`.

In [14]:
date_sta = datetime(2022,10,1)
date_end = datetime(2022,11,1)

infilenames_list = []
outfilenames_list = []

datetimes=generate_datetimes_months(date_sta,date_end,interval=1)
for dt in datetimes:
    date_YM=dt.strftime('%Y%m')
    # zs
    zsfile = f"/scratch/nf33/{user}/hk25-AusCyclones/data/zs_era5_oper_sfc_invariant.nc"
    # msl
    mslfile = get_GADI_ERA5_filename('msl',dt,stream='hourly',level_type='single-levels')
    # z
    zfile = get_GADI_ERA5_filename('z',dt,stream='hourly',level_type='pressure-levels')
    #seaice 
    


    infilenames_list.append(f"{zfile};{zsfile};{mslfile}")
    outfilenames_list.append(f"{detect_dir}/detectNodes_{date_YM}.txt")
        
input_filelist=f"{input_dir}/input_files.txt"
write_to_filelist(infilenames_list,input_filelist)
detect_filelist=f"{detect_dir}/detect_files.txt"
write_to_filelist(outfilenames_list,detect_filelist)
stitch_file=f"{stitch_dir}/stitchNodes.csv"

PermissionError: [Errno 13] Permission denied: '/scratch/nf33/ad1803/hk25-AusCyclones/ETC_workfolder/Test_month/ETCs_Test_2022_10/input/input_files.txt'

**Run TempestExtremes DetectNode**

DetectNode detects nodes  


Thresholds (`closedcontour_commands`) are applied: 

(a) `msl,200.0,5.5,0` represents that mean sea level pressure must increase by 200 Pa over a 5.5 great circle distance (GCD) from the detected node;  


(b) `_DIFF(z(300millibars),z(500millibars)),-58.8,6.5,1.0` represents that the difference between geopotential (Z) on the 300 and 500 millibars surfaces must decrease by 58.8 m2 s−2 over a 6.5 GCD, using the maximum value of this field within 1 GCD as reference. This ensures a coherent upper-level warm core attached to the detected surface low


More details can be found in [Ullrich et al., 2021](https://gmd.copernicus.org/articles/14/5023/2021/)

In [28]:
run_detectNodes(f"{input_dir}/input_files.txt", #input filelist
                f"{detect_dir}/detect_files.txt", #output filelist 
                N_CORES, # cores used for mpi parallel running
                detect_var="msl", # variable used to detect nodes
                merge_dist=6.0,   # merge distance of detected nodes are close to each other of 6.0 great circle distance (GCD)
                closedcontour_commands="msl,200.0,5.5,0",
                output_commands="msl,min,0;zs,min,0",
                timeinterval="6hr",
                lonname="longitude",latname="latitude", 
                logdir=f"{case_dir}/",
                quiet=True,
                out_command_only=True, #Set to true to produce the command line function without running it here (set to false to run in python notebook only)
                )

mpirun -np 1 /scratch/nf33/tempestextremes/bin/DetectNodes --in_data_list /scratch/nf33/ad1803/hk25-AusCyclones/ETC_workfolder/Test_month/ETCs_Test_2022_10/input/input_files.txt --out_file_list /scratch/nf33/ad1803/hk25-AusCyclones/ETC_workfolder/Test_month/ETCs_Test_2022_10/detectNodes/detect_files.txt --searchbymin msl --closedcontourcmd "msl,200.0,5.5,0" --mergedist 6.0 --outputcmd "msl,min,0;zs,min,0" --timefilter "6hr" --latname latitude --lonname longitude --logdir /scratch/nf33/ad1803/hk25-AusCyclones/ETC_workfolder/Test_month/ETCs_Test_2022_10/


We can monitor the algrithm progress through log files under `log_dir`; 40-year TC detection took around 1.5 hrs with 108 nodes.

**Run TempestExtremes StitchNode**

StitchNode connects detected nodes in time.  


Thresholds (`threshold_condition`) are applied:  

(a) `wind,>=,10.0,10` represents that the wind magnitude must be greater than 10 m/s for at least 10 timesteps;  

(b) `lat,<=,50.0,10;lat,>=,-50.0,10` represents that the latitude for detected nodes must be within 50S and 50N for at least 10 timesteps;  

(c) `zs,<,150,10` represents that the detected Node must exit below 150 m for at least 10 timesteps  


More details can be found in [Ullrich et al., 2021](https://gmd.copernicus.org/articles/14/5023/2021/)

In [16]:
# Run TempestExtremes StitchNode
run_stitchNodes(f"{detect_dir}/detect_files.txt", # inputfile list (detectNodes output)
                stitch_file, # output file
                N_CORES, # cores used for mpi parallel running StitchNode run very fast with only one core
                output_filefmt="csv", # output format
                in_fmt_commands="lon,lat,msl,zs", # input format of the detectnode ouput
                range_dist=6.0, # the maximum distance (in GCD) that a node can move between two timesteps
                minim_time="24h", # the minimum lifetime of each track
                maxgap_time="18h", # the maximum duration between two timesteps
                min_endpoint_dist=0.0, # the total distance from the start to the end of the trajectory (0.0)
                threshold_condition="zs,<=,50,1", # threshold, set threshold for orography
                quiet=True,
                out_command_only=True)

mpirun -np 12 /scratch/nf33/tempestextremes/bin/StitchNodes --in_list /scratch/nf33/ad1803/hk25-AusCyclones/ETC_workfolder/ETCs_Climatology/detectNodes/detect_files.txt --in_fmt "lon,lat,msl,zs" --range 6.0 --mintime 24h --maxgap 18h --threshold "zs,<=,50,1" --min_endpoint_dist 0.0 --out_file_format csv --out /scratch/nf33/ad1803/hk25-AusCyclones/ETC_workfolder/ETCs_Climatology/stitchNodes/stitchNodes.csv


# Climatology